# dbitseq

In [1]:
import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd


import seaborn as sns
from collections import Counter
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score

from src.interface import UnitedNet
from src.configs import *
from scipy.stats import spearmanr, pearsonr

In [2]:
def split_data(test_batch):
    adata_rna_train = adata_rna_all[adata_rna_all.obs['batch'] != test_batch]
    adata_morph_train = adata_morph_all[adata_morph_all.obs['batch'] != test_batch]
    adata_mrna_niche_train = adata_mrna_niche_all[adata_mrna_niche_all.obs['batch'] != test_batch]

    adata_rna_test = adata_rna_all[adata_rna_all.obs['batch'] == test_batch]
    adata_morph_test = adata_morph_all[adata_morph_all.obs['batch'] == test_batch]
    adata_mrna_niche_test = adata_mrna_niche_all[adata_mrna_niche_all.obs['batch'] == test_batch]

    return [adata_rna_train, adata_morph_train, adata_mrna_niche_train], [adata_rna_test, adata_morph_test,
                                                                          adata_mrna_niche_test]


def concat_adatas(adatas_train, adatas_test):
    return [ad.concat([adata_train, adata_test]) for adata_train, adata_test in zip(adatas_train, adatas_test)]


def save_umap(adata_all,label,test_batch,nametype):
  fig,ax = plt.subplots(figsize=(6,4))
  ax=sc.pl.umap(adata_all,color=label, ax=ax,show=False)
  fig.savefig(root_save_path+f'/plot/{test_batch}_{nametype}_{label}.png',dpi=300)
  

def generate_adata(data, nonnan_indices, cell_type_label, cols, rows, batch):
    data = data.loc[data.index[nonnan_indices]]
    adata=ad.AnnData(X=np.array(data),obs=list(data.index))
    adata.obs['label']  = cell_type_label
    adata.obs['imagecol'] = cols
    adata.obs['imagerow'] = rows
    adata.obs['batch']  = batch
    return adata

In [3]:
from sklearn import preprocessing
def change_label(adata,batch):
    adata.obs['batch'] = batch
    adata.obs['imagecol'] = adata.obs['array_col']
    adata.obs['imagerow'] = adata.obs['array_row']
    adata.obs['label'] = adata.obs['cell_type']
    return adata

In [4]:
def pre_ps(adata_list,sc_pre = None):
    adata_list_all = [ad_x.copy() for ad_x in adata_list]
    scalars = []
    assert (adata_list_all[0].X>=0).all(), "poluted input"
    for idx, mod in enumerate(adata_list_all):
        t_x = mod.X
        if sc_pre != None:
            scaler = sc_pre[idx]
        else:
            scaler = preprocessing.StandardScaler().fit(t_x)
        t_x = scaler.transform(t_x)
        mod.X = t_x
        adata_list_all[idx] = mod
        scalars.append(scaler)

    return adata_list_all,scalars

# load dbitseq data

In [5]:
technique = 'dbitseq'
data_path = f"../data/{technique}"
device = "cuda:0"

In [6]:
adata_niche_rna_train = sc.read_h5ad(f'{data_path}/adata_niche_rna_train.h5ad')
adata_niche_rna_test = sc.read_h5ad(f'{data_path}/adata_niche_rna_test.h5ad')

adata_rna_train = sc.read_h5ad(f'{data_path}/adata_rna_train.h5ad')
adata_rna_test = sc.read_h5ad(f'{data_path}/adata_rna_test.h5ad')

adata_protein_train = sc.read_h5ad(f'{data_path}/adata_protein_train.h5ad')
adata_protein_test = sc.read_h5ad(f'{data_path}/adata_protein_test.h5ad')


In [7]:
adata_rna_train = change_label(adata_rna_train,'train')
adata_protein_train=change_label(adata_protein_train,'train')
adata_niche_rna_train=change_label(adata_niche_rna_train,'train')

adata_rna_test = change_label(adata_rna_test,'test')
adata_protein_test = change_label(adata_protein_test,'test')
adata_niche_rna_test = change_label(adata_niche_rna_test,'test')

adatas_train = [adata_rna_train, adata_protein_train, adata_niche_rna_train]
adatas_test = [adata_rna_test, adata_protein_test, adata_niche_rna_test]

adatas_all = []
for ad_train, ad_test in zip(adatas_train,adatas_test):
    ad_all = ad_train.concatenate(ad_test,batch_key='sample')
    ad_all = change_label(ad_all,'test')
    adatas_all.append(ad_all)
adatas_all,_ = pre_ps(adatas_all)    
    

adatas_train,_ = pre_ps(adatas_train)   
adatas_test,_ = pre_ps(adatas_test)   


C:\Users\xit882\.conda\envs\pytorch\lib\site-packages\anndata\_core\anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


# Train and finetune UnitedNet

In [13]:

if train_model:
    technique = 'dbitseq'
    data_path = f"../data/{technique}"
    root_save_path = f"../saved_results/dlpfc"
    model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)
    model.train(adatas_train,verbose=True)
    model.finetune(adatas_all,verbose=True)
else:
    technique = 'dbitseq'
    data_path = f"../data/{technique}"
    root_save_path = f"../saved_results/dlpfc"
    model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)



training


  0%|                                                                                                                                                                                                           | 0/20 [00:00<?, ?it/s]



Losses                    Value
-------------------  ----------
translation_loss     0.835885
discriminator_loss   0.0500271
generator_loss       0.0238487
contrastive_loss     0.00538122
reconstruction_loss  0.721778
best_head            9
best model saved at ../saved_results/dlpfc/train_best.pt 



  5%|█████████▊                                                                                                                                                                                         | 1/20 [00:10<03:12, 10.15s/it]



Metrics    Value
---------  -------------------------------------------------------
r2         [[ 6.12120218e-04 -1.13636726e-03  6.90314419e-04]
            [ 6.34486624e-01  7.96618184e-01  3.45287377e-01]
            [ 6.66547313e-03 -1.30406882e-03  3.74370256e-02]]
confusion  [[  0   0   0   0  81   0   0   0   0   0   0   0   0]
            [  0   0   0   0  83   0   0   0   0   0   0   0   0]
            [  0   0   0   0 124   0   0   0   0   0   0   0   0]
            [  0   0   0   0 121   0   0   0   0   0   0   0   0]
            [  0   0   0   0 165   0   0   0   0   0   0   0   0]
            [  0   0   0   0  72   0   0   0   0   0   0   0   0]
            [  0   0   0   0  17   0   0   0   0   0   0   0   0]
            [  0   0   0   0   6   0   0   0   0   0   0   0   0]
            [  0   0   0   0   9   0   0   0   0   0   0   0   0]
            [  0   0   0   0  44   0   0   0   0   0   0   0   0]
            [  0   0   0   0  18   0   0   0   0   0   1   0   0]
 

...



Metrics    Value
---------  ------------------------------------------
r2         [[0.37994621 0.29175418 0.34923913]
            [0.95406307 0.96415223 0.93555384]
            [0.77981778 0.63743252 0.79961205]]
confusion  [[39  0  0  0  0  6  5  4 10 34  0  0  3]
            [ 4 37  0  1  1 20  4 23  3  0  2  0  9]
            [ 1 17 34 31  0  4 31 14  1  0 18  0  4]
            [ 7  2 46 55  0  0  1  1 10  4 22  0  3]
            [16  0  0  2 68 48 16 17  0  1  1  0 38]
            [ 0 17  0  3 28 21  1  7  1  0  2  0 10]
            [ 0  0  0  0  0  0  1  0  4 11  5  0  0]
            [ 0  0  0  0  0  4  0  2  0  0  0  0  1]
            [ 1  3  0  0  0  0  0  0  6  1  1  0  0]
            [ 4  0  0  0  0  0  0  0 23 28  0  0  0]
            [ 4  0  0  8  0  0  0  0  1  5  5  0  0]
            [ 0  0  0  0  0  0  1  1  3  1  0  0  0]
            [ 0  0  0  0  0  2  1  0  0  0  0  0  1]]
acc        0.3173076923076923
ari        0.18395436692139916
nmi        0.36037179749781373


L

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎         | 19/20 [03:47<00:11, 11.88s/it]



Metrics    Value
---------  ------------------------------------------
r2         [[0.39156531 0.30274174 0.35041289]
            [0.95125578 0.96526743 0.92332786]
            [0.78908377 0.66023317 0.80338019]]
confusion  [[45  0  0  0  0  7  0  6 11 27  0  4  1]
            [ 5 37  0  0  1 18  3 25  3  0  0  3  9]
            [ 1 17 42 28  0  5 19 14  1  0  0 26  2]
            [ 6  2 52 50  0  0 22  1 11  3  0  1  3]
            [16  0  0  2 67 51  1 18  0  1  0 15 36]
            [ 0 18  0  3 29 22  2  7  0  0  0  1  8]
            [ 1  0  0  0  0  0  6  0  6  8  0  0  0]
            [ 0  0  0  0  0  4  0  2  0  0  0  0  1]
            [ 1  3  0  0  0  0  0  0  6  1  0  0  1]
            [ 4  0  0  0  0  0  0  0 23 28  0  0  0]
            [ 4  0  0  9  0  0  5  0  1  4  0  0  0]
            [ 1  0  0  0  0  0  0  1  3  1  0  0  0]
            [ 0  0  0  0  0  2  0  1  0  0  0  0  1]]
acc        0.3269230769230769
ari        0.18795034145774991
nmi        0.36710068077046043


L

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [03:59<00:00, 11.96s/it]



Metrics    Value
---------  ------------------------------------------
r2         [[0.39842969 0.2967907  0.36261854]
            [0.95700692 0.96644045 0.93694612]
            [0.79665619 0.6456391  0.80570573]]
confusion  [[46  0  0  0  0  7  0  4 10 27  0  5  2]
            [ 5 37  0  0  1 17  4 26  3  0  0  3  8]
            [ 2 17 39 28  0  3 19 15  1  0  0 29  2]
            [ 7  2 51 49  1  0 23  1 10  3  0  1  3]
            [19  0  0  2 67 49  2 17  0  0  0 16 35]
            [ 0 18  0  3 29 22  3  6  0  0  0  2  7]
            [ 0  0  0  0  0  0  7  0  6  8  0  0  0]
            [ 0  0  0  0  0  4  0  2  0  0  0  0  1]
            [ 1  2  0  0  0  0  1  0  7  1  0  0  0]
            [ 4  0  0  0  0  0  0  0 24 27  0  0  0]
            [ 5  0  0  7  0  0  6  0  1  4  0  0  0]
            [ 0  0  0  0  0  0  0  1  3  1  0  1  0]
            [ 0  0  0  0  0  2  0  1  0  0  0  0  1]]
acc        0.32585470085470086
ari        0.18584140966288323
nmi        0.3649860078221325


# Predict on all data

In [38]:

model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)
model.load_model(f"{root_save_path}/train_best.pt",device=torch.device(device))
model.model.device_in_use = device
# model.evaluate(adatas_train,give_losses=False,stage="train")
predict_label = model.predict_label(adatas_all)

print(root_save_path,'ari:',adjusted_rand_score(adatas_all[0].obs['cell_type'],
                    predict_label))

../saved_results/dlpfc_568_dim_22_0.001_0_None_64_16_16_15_15_False_normed ari: 0.26129926394326425


In [26]:
from src.data import create_dataloader
dataloader_test = create_dataloader(
    model.model,
    adatas_train,
    shuffle=False,
    batch_size=model.model.config["train_batch_size"],
)

In [33]:
import torch
model.model.best_head = torch.tensor(7)

In [34]:
from src.scripts import run_evaluate
metrics = run_evaluate(model.model, dataloader_test)

In [35]:
metrics

{'r2': array([[0.23371273, 0.16326781, 0.2111633 ],
        [0.9213446 , 0.94201426, 0.88849843],
        [0.65070852, 0.45468791, 0.66881152]]),
 'confusion': array([[56,  6,  0,  0,  4,  0,  1,  1,  0, 11,  2,  0,  0],
        [ 0, 21, 32,  0, 14,  0,  1, 10,  0,  4,  1,  0,  0],
        [ 2, 12, 43, 42,  4,  1, 15,  4,  0,  1,  0,  0,  0],
        [ 0,  0,  1, 84,  0,  0, 20,  0,  0, 11,  5,  0,  0],
        [14, 19,  2,  0, 41, 59,  1, 29,  0,  0,  0,  0,  0],
        [ 0,  7, 15,  0, 13, 32,  0,  5,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  9,  0,  0,  2,  6,  0,  0],
        [ 0,  2,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  8,  0,  0,  0],
        [10,  0,  0,  0,  0,  0,  2,  0,  0, 19, 13,  0,  0],
        [ 0,  0,  0,  1,  0,  0,  4,  0,  0,  0, 14,  0,  0],
        [ 1,  1,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0],
        [ 0,  1,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int64),
 'acc': 0.4264705882

In [19]:
#smooth and plot results

coord=np.array((list(adatas_all[0].obs['array_row'].astype('int')),
                list(adatas_all[0].obs['array_col'].astype('int')))).T


united_clus=list(predict_label)

coord=np.array((list(adatas_all[0].obs['array_row'].astype('int')),
                list(adatas_all[0].obs['array_col'].astype('int')))).T

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(coord)
distances,indices = nbrs.kneighbors(coord)

united_clus_new=[]
for indi,i in enumerate(united_clus):
  np.array(united_clus)[(indices[indi])]
  occurence_count=Counter(np.array(united_clus)[(indices[indi])])
  united_clus_new.append(occurence_count.most_common(1)[0][0])
    

cluster_pl = sns.color_palette('tab20',20)
color_list = [cluster_pl[5],
 cluster_pl[1],
 cluster_pl[2],
 cluster_pl[4],
 cluster_pl[11],
 cluster_pl[6],
 cluster_pl[3],
 cluster_pl[7],
 cluster_pl[8],
 cluster_pl[0]]    

In [1]:
plt.figure(figsize=(6,5))
for idx,clus_id in enumerate(set(united_clus_new)):
    
    plt.scatter(adatas_all[0].obs['array_row'][(united_clus_new==clus_id)],
               adatas_all[0].obs['array_col'][(united_clus_new==clus_id)],
               color=color_list[idx],cmap='tab20')
plt.axis('off')
# plt.savefig('dbitseq.png',dpi=300)